In [4]:
import os

#Loading Togehter.ai API
TOGETHER_API_KEY = os.environ["TOGETHER_API_KEY"]

In [27]:
import pandas as pd
import numpy as np
import random
from faker import Faker
from langchain_together import Together

# Initialize Faker for generating random user and beer names
fake = Faker()

# Initialize the language model for generating reviews
llm = Together(
    model="teknium/OpenHermes-2p5-Mistral-7B",
    temperature=0.3,
    max_tokens=256,
    top_k=50,
    # together_api_key="YOUR_API_KEY"
)

def generate_beer_review(style):
    """ Generates a detailed beer review using the Together AI model. """
    prompt = f"Generate a detailed beer review for a {style}. Describe its appearance, aroma, taste, and overall impressions in a cohesive manner. Draw inspiration from the following examples:\n\n1. On tap at the Springfield, PA location. Poured a deep and cloudy orange (almost a copper) color with a small sized off white head. Aromas of oranges and all around citric. Tastes of oranges, light caramel, and a very light grapefruit finish. Light lacing left on the glass.\n\n2. On tap at the John Harvards in Springfield PA. Pours a ruby red amber with a medium off white creamy head that left light lacing. Aroma of orange and various other citrus. Flavor of pine, orange, grapefruit, and some malt balance. Very light bitterness for the 80+ IBUs they said this one had.\n\n3. UPDATED: FEB 19, 2003 Springfield, PA. I've never had the Budvar Cristal but this is exactly what I imagined it to be. A clean and refreshing, hoppy beer, medium-bodied with plenty of flavor. This beer's only downfall is an unpleasant bitterness in the aftertaste.\n\n4. On tap at the Springfield PA location billed as the 'Fancy Lawnmower Light'. Pours a translucent clear yellow with a small bubbly white head. Aroma was lightly sweet and malty, really no hop presence. Flavor was light, grainy, grassy, and malty. Just really light in flavor and aroma overall. Watery.\n\n5. On tap at the Springfield, PA location. Poured a lighter golden color with a very small, if any head. Aromas and tastes of grain, very lightly fruity with a light grassy finish. Lively yet thin and watery body. Oh yeah, the person seating me told me this was a new one and was a Pale Ale even though the menu he gave me listed it as a lighter beer brewed in the Kölsch style.\n\n6. Sampled @ the Springfield, PA location. Candi Sugar dominates this Belgian Ale. Beer was on the flat side but had a nice crimson color. Enjoyable Belgian Ale, I did not expect John Harvards to have it in its lineup.\n\n7. Springfield... Poured a hazy copper color with a medium-sized, off white head that left spotty lacing on the glass. Aroma of yeast, dried fruits, clove, banana, and cherries, with light roastiness. Aroma was very dubbelish. Herbal taste of dark fruits, yeast, and alcohol was barely noticed. Slick mouthfeel. Could have been more flavorful.\n\n8. UPDATED: FEB 19, 2003 Springfield, PA. Darkish copper colored, with no head - probably poured like that on purpose. Served inappropriately at about 40 deg F. This beer was cold. It tasted fine at that temp but I had to let it warm up for a while. It was worth the wait, as a very interesting and complex character developed. Very phenolic and funky - with a strong ester of bubblegum. Also a little clove or some kind of spice. Strong but not overwhelming at all. Surprisingly easy to drink.\n\n9. UPDATED: FEB 19, 2003 Springfield, PA. Sharp and cloyingly sweet. The alcohol presence becomes more and more noticeable. From Springfield PA: nice smooth malty flavor, mildly fruity, but served via nitro and thru a restrictor disc (stout tap). Thus, overly creamy and lacking some of its original flavor. I could tell there was a pretty good beer in there. Aroma difficult to detect.\n\nProvide your own detailed review, focusing on key aspects like appearance, aroma, taste, and overall impressions."
    response = llm.invoke(prompt)
    return response

def generate_beers(n):
    """ Generates a list of beers with random attributes. """
    return [{'beer_name': fake.word(),
             'beer_beerid': fake.uuid4(),
             'beer_brewerid': random.randint(1000, 9999),
             'beer_abv': round(random.uniform(3.0, 12.0), 1),
             'beer_style': random.choice(['IPA', 'Stout', 'Lager', 'Ale', 'Porter'])} for _ in range(n)]

def generate_reviewer_profiles(n):
    """ Generates a list of reviewer profiles. """
    return [{'reviewer_id': fake.uuid4(), 'username': fake.user_name()} for _ in range(n)]

def generate_reviews(beers, reviewer_profiles, num_reviews):
    """ Generates a DataFrame containing reviews for the beers. """
    reviews = []
    for _ in range(num_reviews):
        beer = random.choice(beers)
        reviewer = random.choice(reviewer_profiles)
        review_text = generate_beer_review(beer['beer_style'])
        reviews.append({
            'beer_name': beer['beer_name'],
            'beer_beerid': beer['beer_beerid'],
            'beer_brewerid': beer['beer_brewerid'],
            'beer_abv': beer['beer_abv'],
            'beer_style': beer['beer_style'],
            'review_appearance': np.random.randint(1, 10),
            'review_aroma': np.random.randint(1, 10),
            'review_palate': np.random.randint(1, 10),
            'review_taste': np.random.randint(1, 10),
            'review_overall': np.random.randint(1, 10),
            'review_time': fake.date_time_this_year().strftime('%Y-%m-%d %H:%M:%S'),
            'review_profilename': reviewer['username'],
            'review_text': review_text
        })
    return pd.DataFrame(reviews)

# Example usage
num_beers = 10
num_reviewers = 10
num_reviews = 10

beers = generate_beers(num_beers)
reviewer_profiles = generate_reviewer_profiles(num_reviewers)
reviews_df = generate_reviews(beers, reviewer_profiles, num_reviews)

print(reviews_df.head())


  beer_name                           beer_beerid  beer_brewerid  beer_abv  \
0     three  3ea70248-9bf3-4e9c-869c-2f71a7e48057           1029       8.1   
1   special  2c8327f1-61eb-4f50-8cb6-8df955ffbd20           8537       7.3   
2   country  b5728932-fe11-4b33-89b3-d96bf5db6109           2889       3.4   
3     three  3ea70248-9bf3-4e9c-869c-2f71a7e48057           1029       8.1   
4    sister  1ff9a8be-0d00-4ecb-8011-80fa58214d81           8730       7.6   

  beer_style  review_appearance  review_aroma  review_palate  review_taste  \
0      Lager                  1             6              6             3   
1     Porter                  2             2              4             8   
2      Stout                  5             8              9             5   
3      Lager                  6             5              3             9   
4        Ale                  4             8              6             1   

   review_overall          review_time review_profilename  \
0

In [31]:
# Inspecting the results 
for review in reviews_df['review_text'][:10]:
    print(review)
    print('\n')



Appearance: The lager pours a pale golden color with a small, frothy white head that leaves some lacing on the glass. It's clear and bright, with good carbonation visible.

Aroma: The aroma of this lager is subtle but pleasant, featuring hints of grain, bread, and a slight floral note. No overpowering hoppiness or fruity esters are present.

Taste: The taste of this lager is crisp and refreshing, with a light malt sweetness balanced by a clean, bitter finish. There's a hint of citrus and a touch of breadiness in the background, but overall it's a simple, straightforward beer that lets the malt and hops shine through.

Overall Impressions: This lager is a great example of the style - clean, crisp, and easy-drinking. It would be perfect for sipping on a hot summer day or pairing with light, flavorful foods. While it may not have the complexity of some craft beers, it's a solid, well-made brew that delivers on its promises. I'd definitely recommend it to anyone looking for a




Beer Re